
2. O Conway’s Game of Life é um exemplo bastante conhecido de um autómato celular . Neste problema vamos modificar as regras do autómato da seguinte forma

    1. O espaço de estados é finito definido por uma grelha de células booleanas (morta=0/viva=1) de dimensão $\,N\times N\,$ (com $N>3$) identificadas por índices $\,(i,j)\in \{1..N\}$.  Estas $\;N^2\;$ células são aqui referidas como “normais”. 
        No estado inicial todas as células normais estão mortas excepto  um quadrado $\,3\times 3\,$, designado por “centro”, aleatoriamente posicionado formado apenas por células vivas.
    2. Adicionalmente existem $\,2\,N+1\,$ “células da borda” que correspondem a um dos índices, $i$ ou $j$, ser zero. As células da borda têm valores constantes que, no estado inicial, são gerados aleatoriamente com uma probabilidade $\,\rho\,$ de estarem vivas.
    3. As células normais o autómato modificam o estado de acordo com a regra “B3/S23”: i.e. a célula nasce (passa de $0$ a $1$) se tem exatamente 3 vizinhos vivos e sobrevive (mantém-se viva) se o número de vizinhos vivos é 2 ou 3, caso contrário morre ou continua morta.

A célula $\;(i_0,j_0)\;$e $\,(i_1,j_1)\,$ são vizinhas sse $\;(i_0-i_1=\pm 1) \,\lor\, (j_0-j_1 = \pm 1)$

Pretende-se:

Construir uma máquina de estados finita que represente este autómato; são parâmetros do problema os parâmetros $\,N,\rho\,$ e a posição do  “centro”.

Verificar se se conseguem provar as seguintes propriedades:
   1. Todos os estados acessíveis contém pelo menos uma célula viva.
   2. Toda a célula normal está viva pelo menos uma vez em algum estado acessível.


In [1]:
#from pysmt.shortcuts import *
from z3 import *

In [2]:
def declare(t, N):
    state = {}
    state['pc'] = Int('pc'+str(t))
    
    state['pos'] = {}
    for i in range(N):
        for j in range(N):
            state['pos'][f'({i},{j})'] = Bool(f'pos({i},{j})'+str(t))


    return state

In [3]:
'''
from pyprobs import Probability as pr
probabilidade de sair True = 25%
pr.prob("25%", num=5)
[False, False, True, False, False]
'''
from pyprobs import Probability as pr
from random import randint

def init(state, prob, N):
    init_state = {}

    center_square = (randint(1, N-2), randint(1, N-2))
    
    #center_square = (1,2)
    
    # colocar o quadrado a True
    for i in range(center_square[0]-1, center_square[0]+2):
        for j in range(center_square[1]-1, center_square[1]+2):
            init_state[(i,j)] = (state['pos'][f'({i},{j})'] == True)
    
    
    # colocar tudo o que não é borda a falso
    for i in range(1, N):
        for j in range(1, N):
            if (i,j) not in init_state:
                init_state[(i,j)] = (state['pos'][f'({i},{j})'] == False)
    
      
    # border-x
    for i in range(0,1):
        for j in range(0, N):
            if (i,j) not in init_state:
                init_state[(i,j)]=(state['pos'][f'({i},{j})'] == pr.prob(prob, num=1))
                
    # border-y
    for i in range(0,N):
        for j in range(0, 1):
            if (i,j) not in init_state:
                init_state[(i,j)]=(state['pos'][f'({i},{j})'] == pr.prob(prob, num=1))
                

    statements = list(init_state.values())
    print(And(state['pc'] == 0, And(statements)))
    return And(
        state['pc'] == 0,
        And(statements)
    )



In [4]:
def vizinhos(i, j, N):
    vizinhos = []
    
    for y in range(i-1, i+2):
        for x in range(j-1, j+2):
            if  0 <= y < N and 0 <= x < N and (y,x) != (i,j):
                vizinhos.append((y,x))
    return vizinhos
            
    
#print(vizinhos(0,0,5))
def trans(curr, prox, N):
    rules = And([ 
        Or(
                And(
                    sum([ If(curr['pos'][f'({v[0]},{v[1]})'], 1, 0) for v in vizinhos(i,j,N) ]) == 3,
                    curr['pos'][f'({i},{j})'] == False,
                    prox['pos'][f'({i},{j})'] == True
                ),
                And(
                    Or(
                        sum([ If(curr['pos'][f'({v[0]},{v[1]})'], 1, 0) for v in vizinhos(i,j,N) ]) == 2,
                        sum([ If(curr['pos'][f'({v[0]},{v[1]})'], 1, 0) for v in vizinhos(i,j,N) ]) == 3,
                    ),
                    curr['pos'][f'({i},{j})'] == True, 
                    prox['pos'][f'({i},{j})'] == True
                ),
                And(
                    Or(
                        sum([ If(curr['pos'][f'({v[0]},{v[1]})'], 1, 0) for v in vizinhos(i,j,N) ]) < 2,
                        sum([ If(curr['pos'][f'({v[0]},{v[1]})'], 1, 0) for v in vizinhos(i,j,N) ]) > 3
                    ),
                    prox['pos'][f'({i},{j})'] == False
                )
            ) 
             for i in range(0, N)
             for j in range(0, N)])
    #print(rules)
    # 0 -> 1
    t0 = And(
        curr['pc'] == 0,
        prox['pc'] == 1,
        rules
    )
    
    
    t1 = And(
        Or([curr['pos'][f'({i},{j})'] for i in range(0,N) for j in range(0,N)]) == True,
        curr['pc'] == 1,
        prox['pc'] == curr['pc'],
        rules
    )

    
    t2 = And(
        Or([curr['pos'][f'({i},{j})'] for i in range(0,N) for j in range(0,N)]) == False,
        And([prox['pos'][f'({i},{j})'] == False for i in range(0,N) for j in range(0,N)]),
        curr['pc'] == 1,
        prox['pc'] == 2
    )
    
    stop = And(
        curr['pc'] == 2,
        prox['pc'] == curr['pc'],
        And([curr['pos'][f'({i},{j})'] == False for i in range(0, N) for j in range(0, N)]),
        And([prox['pos'][f'({i},{j})'] == False for i in range(0, N) for j in range(0, N)])
    )
    
    return Or(t0, t1, t2, stop)
    

In [5]:
def teste():
    s = Solver()
    t = {
        'pos': {
            (0,0): Bool('x'),
            (1,0): Bool('y') == True,
            (2,0): Bool('z') 
    
        }
    }
    
    res = Bool('y') == True
    
    res_bools = If(t['pos'][(1,0)], 1, 0)
    
    s.add(res == res_bools)
    
    #solve(res_bools)
    
    #s.add(If(Implies(x, y, y == And(z,Not(z)))  , 0, 1))
    
    if s.check() == sat:
        print(s)
        print(s.model())
        
    
    
    return 
teste()

[If(y == True, 1, 0) == If(y == True, 1, 0)]
[]


In [6]:
def gera_traco(declare,init,k, N):

    s = Solver()
    
    trace = [declare(i, N) for i in range(k)]

    # adicionar o estado inicial
    s.add(init(trace[0],"25%",N))
    
    # adicionar as transições
    for i in range(k - 1):
        s.add(trans(trace[i], trace[i+1], N))
    
    
    check = s.check()
    if check == sat:
        m = s.model()
        print(m)
        for i in range(k):
            for v in trace[i]:
                if v == 'pos':
                    for k in range(0, N):
                        for j in range(0, N):
                            if m[trace[i][v][f'({k},{j})']] == False:
                                print("F", end=" ")
                            else:
                                print("T", end=" ")
                        print("\n")        
                    #        print(v, f"({k},{j})=", m[trace[i][v][f'({k},{j})']])
                else:
                    print(v, "=", m[trace[i][v]])
            print("----------------")
    else:
        print(check)
                
gera_traco(declare,init,5, 4)

And(pc0 == 0,
    And(pos(1,0)0 == True,
        pos(1,1)0 == True,
        pos(1,2)0 == True,
        pos(2,0)0 == True,
        pos(2,1)0 == True,
        pos(2,2)0 == True,
        pos(3,0)0 == True,
        pos(3,1)0 == True,
        pos(3,2)0 == True,
        pos(1,3)0 == False,
        pos(2,3)0 == False,
        pos(3,3)0 == False,
        pos(0,0)0 == False,
        pos(0,1)0 == False,
        pos(0,2)0 == False,
        pos(0,3)0 == False))
unsat
